In [9]:
import pandas as pd
import numpy as np
from thefuzz import fuzz
from thefuzz import process

# 6. Carregar o dataset - Usando o arquivo CSV da planilha Transactions
# O skiprows=1 é usado para pular a linha de metadados antes do cabeçalho real

from google.colab import drive
drive.mount('/content/drive')
path_completo = '/content/drive/MyDrive/Atividade 14.10/Atividade2/archive (1)/KPMG_VI_New_raw_data_update_final.xlsx'
df = pd.read_excel(path_completo, sheet_name='Transactions', header=1)
df_original = df.copy()

print(f"--- Dataset Carregado ---")
print(f"Tamanho original: {len(df)}")
print("-" * 40)

# 7. Identificar e remover duplicatas exatas
exact_duplicates = df.duplicated().sum()
print(f"Número de duplicatas exatas: {exact_duplicates}")

df_no_exact_duplicates = df.drop_duplicates()
removed_exact = len(df) - len(df_no_exact_duplicates)
print(f"Registros removidos (exatas): {removed_exact}")
print(f"Tamanho após remoção de duplicatas exatas: {len(df_no_exact_duplicates)}")
print("-" * 40)


# 8. Identificar duplicatas aproximadas usando:

# Dica: Criar uma função para comparar strings usando fuzzywuzzy
def fuzzy_match(str1, str2, threshold=80):
    """Retorna True se a similaridade for maior ou igual ao threshold."""
    if pd.isnull(str1) or pd.isnull(str2):
        return False
    return fuzz.ratio(str(str1).lower(), str(str2).lower()) >= threshold

# Colunas a serem consideradas na detecção de duplicatas aproximadas
# Exemplo: 'brand' e 'product_line' para produtos similares
cols_for_approx = ['brand', 'product_line', 'product_class', 'product_size']
df_approx = df_no_exact_duplicates.sort_values(by=cols_for_approx).reset_index(drop=True)

approx_duplicates_indices = set()
df_approx_copy = df_approx.copy()
n_rows = len(df_approx_copy)

# Implementação simplificada de comparação de strings para uma coluna-chave (ex: 'brand')
print("--- 8. Detecção de Duplicatas Aproximadas (Exemplo em 'brand') ---")
for i in range(n_rows):
    # Evita reprocessar uma linha já identificada como duplicata
    if i in approx_duplicates_indices:
        continue

    current_brand = df_approx_copy.loc[i, 'brand']

    # Compara a linha atual com as próximas 100 linhas (para otimizar o tempo)
    for j in range(i + 1, min(i + 100, n_rows)):
        other_brand = df_approx_copy.loc[j, 'brand']

        # Verifica a similaridade na coluna 'brand'
        if fuzzy_match(current_brand, other_brand, threshold=95):
            # Se a marca for muito similar, consideramos uma possível duplicata aproximada
            approx_duplicates_indices.add(j)

approx_duplicates_count = len(approx_duplicates_indices)
print(f"Número de duplicatas aproximadas (estimado na coluna 'brand' com similaridade > 95%): {approx_duplicates_count}")
print("-" * 40)

# 9. Estratégia para lidar com duplicatas aproximadas (Manter a primeira ocorrência)

# Crie uma coluna de identificador para as duplicatas aproximadas
# Nota: Esta é uma estratégia simplificada. Em um caso real, você criaria um 'cluster' de duplicatas.
df_approx_identified = df_approx_copy.iloc[list(set(df_approx_copy.index) - approx_duplicates_indices)].copy()
df_approx_removed = df_approx_copy.iloc[list(approx_duplicates_indices)]

# Estratégia: Manter a primeira ocorrência de um grupo de "quase-duplicatas"
# Como simplificação, removemos todas as linhas identificadas como "próximas" da lista.
# Em um pipeline real, você combinaria informações ou manteria o registro mais recente (max(transaction_date)).
df_cleaned_final = df_approx_identified.copy()

print("--- 9. Estratégia Implementada: Remoção (Manter a primeira ocorrência) ---")
print(f"Tamanho do dataset após tratar duplicatas aproximadas: {len(df_cleaned_final)}")
print("-" * 40)

# 10. Documentar o impacto
total_removed = removed_exact + approx_duplicates_count # Apenas uma estimativa
print("--- 10. Documentação do Impacto ---")
print(f"Total de registros originais: {len(df_original)}")
print(f"Registros removidos (duplicatas exatas): {removed_exact}")
print(f"Registros removidos (duplicatas aproximadas - estimativa): {approx_duplicates_count}")
print(f"Total de registros no dataset final: {len(df_cleaned_final)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--- Dataset Carregado ---
Tamanho original: 20000
----------------------------------------
Número de duplicatas exatas: 0
Registros removidos (exatas): 0
Tamanho após remoção de duplicatas exatas: 20000
----------------------------------------
--- 8. Detecção de Duplicatas Aproximadas (Exemplo em 'brand') ---
Número de duplicatas aproximadas (estimado na coluna 'brand' com similaridade > 95%): 19602
----------------------------------------
--- 9. Estratégia Implementada: Remoção (Manter a primeira ocorrência) ---
Tamanho do dataset após tratar duplicatas aproximadas: 398
----------------------------------------
--- 10. Documentação do Impacto ---
Total de registros originais: 20000
Registros removidos (duplicatas exatas): 0
Registros removidos (duplicatas aproximadas - estimativa): 19602
Total de registros no dataset final: 398


In [4]:
!pip install thefuzz[speedup]